In [ ]:
!conda install -y gdown
!gdown 1i6gsJch1_Odnz-ddqGQzpakD5mowwx-6

In [ ]:
!apt install -y python-opengl ffmpeg > /dev/null 2>&1

# !apt install -y xvfb
%pip install pyvirtualdisplay
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

In [ ]:
!pip install mujoco
!pip install mujoco_mjx
!pip install brax

In [ ]:
#@title Check if MuJoCo installation was successful
import os


# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=glfw

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Tell XLA to use Triton GEMM, this improves steps/sec by ~30% on some GPUs
os.environ['XLA_FLAGS'] = (
    '--xla_gpu_enable_triton_softmax_fusion=true '
    '--xla_gpu_triton_gemm_any=True '
    '--xla_gpu_enable_async_collectives=true '
    '--xla_gpu_enable_latency_hiding_scheduler=true '
    '--xla_gpu_enable_highest_priority_async_stream=true '
)


In [ ]:
!sudo apt install unrar
!pip install patool
import patoolib
patoolib.extract_archive("/kaggle/working/Spider.rar",outdir='/kaggle/working/')

In [ ]:
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
import random
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
# !command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [ ]:
#@title Import MuJoCo, MJX, and Brax
from datetime import datetime
from etils import epath
import functools
from IPython.display import HTML
from typing import Any, Dict, Sequence, Tuple, Union
import os
from ml_collections import config_dict
import math

import jax
from jax import numpy as jp
from flax.training import orbax_utils
from flax import struct
import mediapy as media
from orbax import checkpoint as ocp

import mujoco
from mujoco import mjx

from brax import base
from brax import envs
from brax import math as braxMath
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.es import train as es
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.es import networks as es_networks
from brax.io import html, mjcf, model
from jax.experimental.ode import odeint


In [ ]:
def domain_randomize(sys, rng):
  """Randomizes the mjx.Model."""
  @jax.vmap
  def rand(rng):
    _, key = jax.random.split(rng, 2)
    # friction
    friction = jax.random.uniform(key, (1,), minval=0.6, maxval=1.4)
    friction = sys.geom_friction.at[:, 0].set(friction)
    # actuator
    _, key = jax.random.split(key, 2)
    gain_range = (-5, 5)
    param = jax.random.uniform(
        key, (1,), minval=gain_range[0], maxval=gain_range[1]
    ) + sys.actuator_gainprm[:, 0]
    gain = sys.actuator_gainprm.at[:, 0].set(param)
    bias = sys.actuator_biasprm.at[:, 1].set(-param)
    return friction, gain, bias

  friction, gain, bias = rand(rng)

  in_axes = jax.tree_util.tree_map(lambda x: None, sys)
  in_axes = in_axes.tree_replace({
      'geom_friction': 0,
      'actuator_gainprm': 0,
      'actuator_biasprm': 0,
  })

  sys = sys.tree_replace({
      'geom_friction': friction,
      'actuator_gainprm': gain,
      'actuator_biasprm': bias,
  })

  return sys, in_axes

In [ ]:
xml_path = r'/kaggle/working/Spider_Assembly_fineMesh_frictionDamp/urdf/final_noFrictionLoss_noCoxaCon_explicitConPair_ellipsoidTibias_kaggle.xml'

class HexapodV0_3(PipelineEnv):
    def __init__(self,
                 xml_path,
                 terminateWhenTilt=True,
                 terminateWhenTiltGreaterThan=40 * jp.pi / 180,
                 baseTiltSigma=0.3,
                 baseTiltCoef=1,
                 
                 kick_size = 0.05,
                 obs_noise=0.05,
                 foot_slip=-0.0,
                 lin_vel_z=-2.0,
                 ang_vel_xy=-0.05,
                 torques=-0.0005,
                 feet_air_time=0.,
                 stand_still=-2,

                 terminateWhenFumersColide=False,
                 femurCollisionSigma=0.06,
                 femurCollisionCoef=0,

                 correctDirectionSigma=0.3,
                 correctDirectionWeight=1.5,
                 deviationAngleSigma=0.3,
                 deviationAngleWeight=1,

                 baseHeightSigma=0.027,
                 baseHeightCoef=1,
                 terminateWhenLow=True,
                 baseHeightLowerLimit=0.10,
                 baseOscillationSigma=0.5,
                 baseOscillationCoef=1.0,

                 rewardForTibiaTip=True,
                 tibiaRewardSigma=0.05,
                 tibiaRewardCoef=0,

                 powerCoef=0.001,
                 continuityCoef=-0.,
                 smoothnessCoef=-0.,

                 includeBaseAngularVels=True,
                 includeTibiaTipSensors=False,
                 nStacks=5,
                 physics_steps_per_control_step=1,
                 CPG_steps=10,

                 resetPosLowHigh=[jp.array([-0.2, -0.2, 0.23]), jp.array([0.2, 0.2, 0.4])],
                 resetOriLowHigh=[jp.array([-jp.pi/12, -jp.pi/12, -jp.pi]), jp.array([jp.pi/12, jp.pi/12, -jp.pi])],
                 resetJointsPosLowHigh = [jp.array([-jp.pi/12]*18), jp.array([jp.pi/12]*18)],
                 resetJointsVelsLowHigh=[jp.array([-0.3]*24), jp.array([-0.3]*24)],
                 **kwargs
                 ):

        self.mj_model = mujoco.MjModel.from_xml_path(xml_path)
        self.mj_model.opt.solver = mujoco.mjtSolver.mjSOL_NEWTON
        self.mj_model.opt.iterations = 6
        self.mj_model.opt.ls_iterations = 6

        sys = mjcf.load_model(self.mj_model)
        sys = sys.tree_replace({'opt.timestep': 0.002})
#         sys = sys.replace(
#         dof_damping=sys.dof_damping.at[6:].set(0.5239)
# #         actuator_gainprm=sys.actuator_gainprm.at[:, 0].set(35.0),
# #         actuator_biasprm=sys.actuator_biasprm.at[:, 1].set(-35.0),
#         )

        kwargs['n_frames'] = kwargs.get(
            'n_frames', physics_steps_per_control_step)
        kwargs['backend'] = 'mjx'
        super().__init__(sys, **kwargs)
        
        #CPG CODE
        self.a = 150
        self.num_joints = 18
        self.lowers_mu = jp.array([0.] * self.num_joints)
        self.uppers_mu = jp.array([1.] * self.num_joints)
        self.lowers_omega = jp.array([0.] * self.num_joints) * 2 * math.pi 
        self.uppers_omega = jp.array([6] * self.num_joints) * 2 * math.pi 
        self.time_step=0.002
        self.CPG_steps = CPG_steps
        ##
        
        tibia_body = [f'tibia{i}' for i in range(1,7)]
        tibia_body_id = [
            mujoco.mj_name2id(sys.mj_model, mujoco.mjtObj.mjOBJ_BODY.value, l)
            for l in tibia_body
        ]
        assert not any(id_ == -1 for id_ in tibia_body_id), 'Body not found.'
        self._tibia_body_id = np.array(tibia_body_id)
        
        tibia_sites = [f'siteFemur{i}Tibia{i}Touch' for i in range(1,7)]
        tibia_sites_id = [
        mujoco.mj_name2id(sys.mj_model, mujoco.mjtObj.mjOBJ_SITE.value, f)
        for f in tibia_sites
        ]
        assert not any(id_ == -1 for id_ in tibia_sites_id), 'Site not found.'
        self._tibia_sites_id = np.array(tibia_sites_id)
        
        self._kick_size = kick_size

        self._obs_noise_coef=obs_noise
        self._foot_slip_coef=foot_slip
        self._lin_vel_z_coef=lin_vel_z
        self._ang_vel_xy_coef=ang_vel_xy
        self._torques_coef=torques
        self._feet_air_time_coef=feet_air_time
        self._stand_still_coef=stand_still
        
        self.action_lower = jp.array([-jp.pi/3]*18)
        self.action_upper = jp.array([jp.pi/3]*18)
        
        self._terminateWhenTilt = terminateWhenTilt
        self._terminateWhenTiltGreaterThan = terminateWhenTiltGreaterThan
        self._baseTiltSigma= baseTiltSigma
        self._baseTiltCoef= baseTiltCoef
        self._terminateWhenFumersColide = terminateWhenFumersColide
        self._femurCollisionSigma = femurCollisionSigma
        self._femurCollisionCoef = femurCollisionCoef
        self._correctDirectionSigma = correctDirectionSigma
        self._correctDirectionWeight = correctDirectionWeight
        self._deviationAngleSigma = deviationAngleSigma
        self._deviationAngleWeight = deviationAngleWeight
        self._baseHeightSigma = baseHeightSigma
        self._baseHeightCoef = baseHeightCoef
        self._terminateWhenLow = terminateWhenLow
        self._baseHeightLowerLimit = baseHeightLowerLimit
        self._baseOscillationSigma = baseOscillationSigma
        self._baseOscillationCoef = baseOscillationCoef
        self._rewardForTibiaTip = rewardForTibiaTip
        self._tibiaRewardSigma = tibiaRewardSigma
        self._tibiaRewardCoef = tibiaRewardCoef
        self._powerCoef = powerCoef
        self._smoothnessCoef = smoothnessCoef
        self._continuityCoef = continuityCoef
        self._includeBaseAngularVels = includeBaseAngularVels
        self._includeTibiaTipSensors = includeTibiaTipSensors
        self._resetPosLowHigh = resetPosLowHigh
        self._resetOriLowHigh = resetOriLowHigh
        self._resetJointsPosLowHigh = resetJointsPosLowHigh
        self._resetJointsVelsLowHigh = resetJointsVelsLowHigh
        self._includeBaseAngularVels = includeBaseAngularVels
        self._includeTibiaTipSensors = includeTibiaTipSensors
        self._nStacks = nStacks
        self._physics_steps_per_control_step = physics_steps_per_control_step

    def reset(self, rng: jp.ndarray) -> State:
        """Resets the environment to an initial state."""
        rng, rng1, rng2, rng3, rng4, numTransitionsRng, desiredVelRng, desiredAngleRng, transitionStepsRng= jax.random.split(rng, 9)

        base_pos = jax.random.uniform(key=rng1, shape=(3,), minval=self._resetPosLowHigh[0], maxval=self._resetPosLowHigh[1])
        # print(base_pos)
        base_orientation_euler = jax.random.uniform(key=rng2, shape=(3,), minval=self._resetOriLowHigh[0],
                                                    maxval=self._resetOriLowHigh[1])
        base_orientation = self._euler_to_quaternion(base_orientation_euler)

        joints_pos = jax.random.uniform(key=rng3, shape=(18,), minval=self._resetJointsPosLowHigh[0],
                                                    maxval=self._resetJointsPosLowHigh[1])
        qpos = jp.concatenate((base_pos, base_orientation, joints_pos), axis=0)

        qvel = jax.random.uniform(key=rng4, shape=(24,), minval=self._resetJointsVelsLowHigh[0],
                                                    maxval=self._resetJointsVelsLowHigh[1])

        data = self.pipeline_init(qpos, qvel)

        obs_history = jp.zeros(self._nStacks * 42)
        reward, done, zero = jp.zeros(3)
        
        num_transitions = random.randint(2, 5)
        desired_vels = jax.random.uniform(key=desiredVelRng, shape=(num_transitions + 1,), minval=0, maxval=1.2)
        desired_angle_vels = jax.random.uniform(key=desiredAngleRng, shape=(num_transitions + 1,), minval=-1, maxval=1)
        transition_steps = jax.random.randint(key=transitionStepsRng, shape=(num_transitions,), minval=50, maxval=951)

        
        state_info = {
            'last_action': jp.zeros(54),
            'num_transitions': num_transitions,
            'desired_vels': desired_vels,
            'desired_angle_vels': desired_angle_vels,
            'transition_steps': transition_steps,
            'current_idx' : 0,
            'step': 0,
            'initial_state': jp.zeros((3, 18)),
            'last_contact': jp.zeros(6, dtype=bool),
            'feet_air_time': jp.zeros(6),
            'rng': rng
        }
        obs= self._get_obs(data, jp.zeros(self.sys.nu), desired_vels[0], desired_angle_vels[0], obs_history, state_info)
        metrics = {
            'correct_direction_reward': zero,
            'base_tilt':zero,
            'base_height': zero,
            'continuity': zero,
            'power': zero,
            'tibia_tip_contact': zero,
            'femur_collision': zero,
            'x_position': zero,
            'y_position': zero,
            'distance_from_origin': zero,
            'x_velocity': zero,
            'y_velocity': zero,
            'base_tilt_reward': zero,
            'tibia_reward': zero,
            'total_reward' : zero
        }
        return State(data, obs, reward, done, metrics, state_info)
    
    @property
    def action_size(self) -> int:
        return 54
    def pipeline_step(self, pipeline_state: Any, action: jax.Array) -> base.State:
        """Takes a physics step using the physics pipeline."""

        def f(state, action_idxed):
          return (
              self._pipeline.step(self.sys, state, action_idxed, self._debug),
              None,
          )

        return jax.lax.scan(f, pipeline_state, action)[0]

    def step(self, state: State, action: jp.ndarray) -> State:
        """Runs one timestep of the environment's dynamics."""
        rng, kick_noise_2 = jax.random.split(state.info['rng'])
        
        # kick
        push_interval = 10
        kick_theta = jax.random.uniform(kick_noise_2, maxval=2 * jp.pi)
        kick = jp.array([jp.cos(kick_theta), jp.sin(kick_theta)])
        kick *= jp.mod(state.info['step'], push_interval) == 0
        qvel = state.pipeline_state.qvel  # pytype: disable=attribute-error
        qvel = qvel.at[:2].set(kick * self._kick_size + qvel[:2])
        state = state.tree_replace({'pipeline_state.qvel': qvel})
        
        prev_pipeline_state = state.pipeline_state
        previous_joint_angels = prev_pipeline_state.q[7:]
        action_reshaped = action.reshape((3, 18))
        mu = action_reshaped[0]
        mu = jp.clip(mu*0.5 + 0.5, a_min=self.lowers_mu, a_max= self.uppers_mu)
        omega = action_reshaped[1]
        omega = jp.clip(omega * 3. * 2 * jp.pi + 3. * 2 * jp.pi, a_min=self.lowers_omega, a_max=self.uppers_omega)
        phase = action_reshaped[2]
        phase = jp.clip(phase * 1 * jp.pi + 1 * jp.pi, a_min=0, a_max=2*jp.pi)
        
        motor_targets_times, new_initial_state = self.get_position(state.info['initial_state'], mu, omega, phase)
        state.info['initial_state'] = state.info['initial_state'].at[:].set(new_initial_state)
        pipeline_state = self.pipeline_step(prev_pipeline_state, motor_targets_times)
        
        current_joint_angels = pipeline_state.q[7:]
        last_action = state.info['last_action']
        desired_vel = state.info['desired_vels'][state.info['current_idx']]
        
        desired_angle_vel = state.info['desired_angle_vels'][state.info['current_idx']]
        # desired_vel *= (desired_vel > 0.15) * (jp.abs(desired_angle_vel) > 0.174)
        # desired_angle_vel *= (jp.abs(desired_angle_vel) > 0.174) * (desired_vel>0.15)

        base_pos = pipeline_state.subtree_com[0]
        velocity = pipeline_state.xd.vel[0, :]

        base_ori = pipeline_state.x.rot[0,:]
        base_tilt = (jp.linalg.norm(self._quaternion_to_euler(base_ori[:])[0:2]))
        base_ang_vel = jp.linalg.norm(pipeline_state.xd.ang[0,0:2])
        
        # Tibia foot contact data based on z-position
        tibia_foot_pos = pipeline_state.site_xpos[self._tibia_sites_id]  # pytype: disable=attribute-error
        tibia_foot_contact_z = tibia_foot_pos[:, 2]
        contact = tibia_foot_contact_z < 1e-3  # a mm or less off the floor
#         contact_filt_mm = contact | state.info['last_contact']
        contact_filt_cm = (tibia_foot_contact_z < 3e-2) | state.info['last_contact']
        first_contact = (state.info['feet_air_time'] > 0) * contact_filt_cm
        state.info['feet_air_time'] += self.dt
        
        # Linear tracking reward
        local_vel = braxMath.rotate(pipeline_state.xd.vel[0], braxMath.quat_inv(pipeline_state.x.rot[0]))
        correctDirectionReward = self._correctDirectionWeight * jp.exp(-(desired_vel - local_vel[0])**2/self._correctDirectionSigma**2) * ((desired_vel > 0.15) | (jp.abs(desired_angle_vel) > 0.174))
        # Angular tracking reward
        base_ang_vel = braxMath.rotate(pipeline_state.xd.ang[0], braxMath.quat_inv(pipeline_state.x.rot[0]))
        correctAngVelReward = 0.8 * jp.exp(-(desired_angle_vel - base_ang_vel[2])**2/self._correctDirectionSigma**2) * ((desired_vel > 0.15) | (jp.abs(desired_angle_vel) > 0.174))
        # Femur collision reward
        femurReward = self._get_femur_reward(pipeline_state)
        # Base tilt reward
        baseTiltReward =  self._baseTiltCoef * jp.exp(-base_tilt**2/self._baseTiltSigma**2)
        # Base height reward
        baseHeightReward = -1* (1 - jp.exp(-(0.25 - pipeline_state.x.pos[0,2])**2/0.2**2) )
        # Foot slip reward
        foot_pos = pipeline_state.site_xpos[self._tibia_sites_id]  # feet position
        feet_offset = foot_pos - pipeline_state.xpos[self._tibia_body_id]
        offset = base.Transform.create(pos=feet_offset)
        foot_indices = self._tibia_body_id - 1  # we got rid of the world body
        foot_vel = offset.vmap().do(pipeline_state.xd.take(foot_indices)).vel
        FootSlipReward = self._foot_slip_coef * jp.sum(jp.square(foot_vel[:, :2]) * contact_filt_cm.reshape((-1, 1)))
        # Base xy angular reward
        baseAngXYReward = self._ang_vel_xy_coef * jp.sum(jp.square(pipeline_state.xd.ang[0, :2]))
        # Torque reward
        torqueReward = self._torques_coef * (jp.sqrt(jp.sum(jp.square(pipeline_state.qfrc_actuator))) + jp.sum(
            jp.abs(pipeline_state.qfrc_actuator)) )
        # Stand still reward
        standStillReward = self._stand_still_coef * jp.sum(jp.abs(pipeline_state.q[7:])) * (
            (desired_vel < 0.15) | (jp.abs(desired_angle_vel) < 0.174))
        # Base linear z reward
        baseZVelreward = self._lin_vel_z_coef * jp.square(pipeline_state.xd.vel[0, 2])
        # tibia air time Reward
        rew_air_time = jp.sum(jp.clip((state.info['feet_air_time'] - 0.5) * first_contact, -math.inf, 1./6.))
        rew_air_time *= (
            (desired_vel > 0.15) | (jp.abs(desired_angle_vel) > 0.174)
        )  
        tibiaAirTimeReward = self._feet_air_time_coef * rew_air_time
        # Action continuity reward
        continuity_reward = self._continuityCoef * (jp.abs(action - last_action)).sum()
        # Smoothness reward
        SmoothnessReward = self._smoothnessCoef * (jp.abs(current_joint_angels - previous_joint_angels)).sum()
        
        # tibiaReward = self._get_tibia_reward(pipeline_state) * self._rewardForTibiaTip
        # baseHeightReward = (self._baseHeightCoef * jp.exp(-(0.23 - base_pos[2])**2/self._baseHeightSigma**2) *
        # (base_pos[2] > self._baseHeightLowerLimit) )
        # baseHeightReward = 0
        # baseOscillationReward = self._baseOscillationCoef * jp.exp(-base_ang_vel**2/self._baseOscillationSigma**2)
        
        termination = jp.array(((base_tilt > self._terminateWhenTiltGreaterThan) |
                       (base_pos[2] < self._baseHeightLowerLimit)), dtype=jp.bool)

        state.info['last_action'] = state.info['last_action'].at[:].set(action)

        rewards = (correctDirectionReward + correctAngVelReward - 5 * termination + baseTiltReward + continuity_reward +
                 SmoothnessReward + 
                 femurReward + FootSlipReward + baseAngXYReward + torqueReward + standStillReward + baseZVelreward +
                 tibiaAirTimeReward + baseHeightReward)
        reward = jp.clip(rewards, 0.0, 10000.0)
        done = 1.0 - ~termination
        # print('1')
        state.info['step'] += 1
        condition = jp.any(state.info['step'] == state.info['transition_steps'])
        new_current_idx = jax.lax.select(condition, state.info['current_idx'] + 1, state.info['current_idx'])
        state.info['current_idx'] = new_current_idx
        state.info['last_contact'] = contact_filt_cm
        state.info['feet_air_time'] *= ~contact_filt_cm
        state.info['rng'] = rng

        new_desired_vel = state.info['desired_vels'][state.info['current_idx']]
        new_desired_angle_vel = state.info['desired_angle_vels'][state.info['current_idx']]
        
        new_desired_vel *= (new_desired_vel > 0.15) * (jp.abs(new_desired_angle_vel) > 0.174)
        new_desired_angle_vel *= (jp.abs(new_desired_angle_vel) > 0.174) * (new_desired_vel>0.15)


        obs = self._get_obs(pipeline_state,  action, new_desired_vel, new_desired_angle_vel, obs_history=state.obs,
                           state_info=state.info)

        state.metrics.update(correct_direction_reward=correctDirectionReward,
                         base_tilt=base_tilt,
                         base_height=base_pos[2],
#                          continuity=continuity_reward,
                         x_position=base_pos[0],
                         y_position=base_pos[1],
                         distance_from_origin=jp.linalg.norm(base_pos[0:2]),
                         x_velocity=velocity[0],
                         y_velocity=velocity[1],
                         base_tilt_reward=baseTiltReward,
#                          tibia_reward=tibiaReward,
#                          total_reward=reward
                             )
        # print(state.metrics)
        return state.replace(
            pipeline_state=pipeline_state, obs=obs, reward=reward, done=done
        )


    def _get_obs(
            self, data: base.State, action: jp.ndarray, desired_vel, desired_angle_vel, obs_history: jax.Array,
        state_info: dict[str, Any]
                ) -> jp.ndarray:
        """Observes"""

        inv_base_rot = braxMath.quat_inv(data.x.rot[0])
        local_rpyrate = braxMath.rotate(data.xd.ang[0], inv_base_rot)
        current_obs = jp.concatenate([jp.array([local_rpyrate[2]]) * 0.25,
                                      braxMath.rotate(jp.array([0, 0, -1]), inv_base_rot),
                                      data.q[7:],
                                      jp.array([desired_vel, desired_angle_vel]) * jp.array([2.0, 0.25]), 
                                     state_info['last_action']],
                                     axis=0)

        # clip, noise
        current_obs = jp.clip(current_obs, a_min=-100.0, a_max=100.0) + self._obs_noise_coef * jax.random.uniform(
        state_info['rng'], current_obs.shape, minval=-1, maxval=1)
        
        obs = jp.roll(obs_history, current_obs.size).at[:current_obs.size].set(current_obs)
        return obs

    def _get_femur_reward(self, pipeline_state):
        femur_dists = pipeline_state.contact.dist[6:]

        contacted = jp.any(femur_dists < 0.01)
        femur_reward = -1 * contacted
        return femur_reward

    def _get_tibia_reward(self, pipeline_state: State) -> jp.ndarray:
        contact_dists = pipeline_state.contact.dist[0:6]
        contact_booleans = (jp.abs(contact_dists) < 0.03) * jp.ones(6)
        tibia_tip_dists = 0
        for i in range(2,8):
            tibia_tip_dists += contact_booleans[i-2] * pipeline_state.site_xpos[i, 2]

        tibia_reward = self._tibiaRewardCoef * jp.exp(-tibia_tip_dists**2/self._tibiaRewardSigma**2)
        return tibia_reward
    
    def cpg_dynamics(self, y, t, a, mu, omega):
        r = y[0]
        r_dot = y[1]
        theta = y[3]
        theta_dot = omega  # Use the omega array directly

        r_ddot = a * (a * (mu - r) / 4 - r_dot)

        return jp.concatenate([r_dot, r_ddot, theta_dot])

    def get_position(self, initial_state, mu, omega, phase):
        t = jp.arange(0, self.CPG_steps * self.time_step, self.time_step)
        solution = odeint(self.cpg_dynamics, initial_state, t, self.a, mu, omega)

        r = solution[:, 0, :]
        r_dot = solution[:, 1, :]
        theta = solution[:, 2, :]
        new_initial_state = jp.vstack((r[-1], r_dot[-1], theta[-1]))

        positions = r * jp.cos(theta + phase)
        return positions, new_initial_state


    def _euler_to_quaternion(self, euler):
        """Converts Euler angles to quaternion."""
        roll, pitch, yaw = euler
        qx = jp.sin(roll/2) * jp.cos(pitch/2) * jp.cos(yaw/2) - jp.cos(roll/2) * jp.sin(pitch/2) * jp.sin(yaw/2)
        qy = jp.cos(roll/2) * jp.sin(pitch/2) * jp.cos(yaw/2) + jp.sin(roll/2) * jp.cos(pitch/2) * jp.sin(yaw/2)
        qz = jp.cos(roll/2) * jp.cos(pitch/2) * jp.sin(yaw/2) - jp.sin(roll/2) * jp.sin(pitch/2) * jp.cos(yaw/2)
        qw = jp.cos(roll/2) * jp.cos(pitch/2) * jp.cos(yaw/2) + jp.sin(roll/2) * jp.sin(pitch/2) * jp.sin(yaw/2)
        return jp.array([qw, qx, qy, qz])

    def _quaternion_to_euler(self, quaternion):
        """Converts quaternion to Euler angles."""
        qw, qx, qy, qz = quaternion

        # Roll (x-axis rotation)
        sinr_cosp = 2 * (qw * qx + qy * qz)
        cosr_cosp = 1 - 2 * (qx * qx + qy * qy)
        roll = jp.arctan2(sinr_cosp, cosr_cosp)

        # Pitch (y-axis rotation)
        sinp = 2 * (qw * qy - qz * qx)
        pitch = jp.where(jp.abs(sinp) >= 1, jp.sign(sinp) * (jp.pi / 2), jp.arcsin(sinp))

        # Yaw (z-axis rotation)
        siny_cosp = 2 * (qw * qz + qx * qy)
        cosy_cosp = 1 - 2 * (qy * qy + qz * qz)
        yaw = jp.arctan2(siny_cosp, cosy_cosp)

        return jp.array([roll, pitch, yaw])

    def render(
            self, trajectory: List[base.State], camera = None,
            width: int = 640, height: int = 480,
            ) -> Sequence[np.ndarray]:
        camera = camera or 'track'
        return super().render(trajectory, camera=camera, width=width, height=height)



env = HexapodV0_3(xml_path=xml_path)


In [ ]:
make_networks_factory = functools.partial(
    ppo_networks.make_ppo_networks,
        policy_hidden_layer_sizes=(128, 128, 128, 128))
train_fn = functools.partial(
      ppo.train, num_timesteps=150_000_000, num_evals=10,
      reward_scaling=1, episode_length=1000, normalize_observations=True,
      action_repeat=1, unroll_length=20, num_minibatches=64,
      num_updates_per_batch=3, discounting=0.99, learning_rate=3.0e-4,
      entropy_cost=1e-4, num_envs=int(8192), batch_size=256,
      network_factory=make_networks_factory,
      randomization_fn=domain_randomize,
      seed=0)

def progress(num_steps, metrics):
  print(metrics['eval/episode_reward'], metrics['eval/episode_reward_std'])
  times.append(datetime.now())
  plt.figure()
  x_data.append(num_steps)
  y_data.append(metrics['eval/episode_reward'])
  ydataerr.append(metrics['eval/episode_reward_std'])

  plt.xlim([0, train_fn.keywords['num_timesteps'] * 1.25])
#   plt.ylim([min_y, max_y])

  plt.xlabel('# environment steps')
  plt.ylabel('reward per episode')
  plt.title(f'y={y_data[-1]:.3f}')

  plt.errorbar(
      x_data, y_data, yerr=ydataerr)
  plt.show()

x_data = []
y_data = []
ydataerr = []
times = [datetime.now()]

env = HexapodV0_3(xml_path=xml_path)
eval_env = HexapodV0_3(xml_path=xml_path)
make_inference_fn, params, metrics = train_fn(environment=env,
                                       progress_fn=progress,
                                       eval_env=eval_env)

print(f'time to jit: {times[1] - times[0]}')
print(f'time to train: {times[-1] - times[1]}')

In [ ]:
model_path = '/kaggle/working/mjx_brax_policy_22'
model.save_params(model_path, params)

In [ ]:
params = model.load_params(model_path)

inference_fn = make_inference_fn(params, deterministic=True)
jit_inference_fn = jax.jit(inference_fn)

In [ ]:
eval_env = HexapodV0_3(xml_path=xml_path, kick_size=0.)

jit_reset = jax.jit(eval_env.reset)
jit_step = jax.jit(eval_env.step)

In [ ]:
rng = jax.random.PRNGKey(0)
state = jit_reset(rng)
rollout = [state.pipeline_state]
rollout_info = [state.info]

# grab a trajectory
n_steps = 1000
render_every = 1

for i in range(n_steps):
  act_rng, rng = jax.random.split(rng)
  ctrl, _ = jit_inference_fn(state.obs, act_rng)
  # for i in range(env._physics_steps_per_control_step):
  state = jit_step(state, ctrl)
  rollout.append(state.pipeline_state)
#   print(state.pipeline_state.xd.ang[0], braxMath.rotate(state.pipeline_state.xd.ang[0], braxMath.quat_inv(state.pipeline_state.x.rot[0])),
#        '\n', state.pipeline_state.x.pos[0],braxMath.rotate(state.pipeline_state.x.pos[0], braxMath.quat_inv(state.pipeline_state.x.rot[0])),
#        '\n************************************************************************')
  rollout_info.append(state.info)

  if state.done:
    break

frames = env.render(rollout, camera='hexapod_camera')
# media.show_video(frames, fps=1.0 / env.dt / 2)

In [ ]:
import cv2


def add_capsule_and_text(frames, desired_vels, desired_angle_vels):
    for i, frame in enumerate(frames):
        desired_vel = desired_vels[i]
        desired_angle_vel = desired_angle_vels[i]

        # Calculate the start and end points of the capsule from the center of the frame
        height, width, _ = frame.shape
        center_x = width // 2
        center_y = height // 2
        capsule_length = int(desired_vel * 100)  # Scale the velocity to pixels
#         angle = desired_angle

#         start_point = (center_x, center_y)
#         end_point = (int(center_x + capsule_length * math.cos(angle)), int(center_y - capsule_length * math.sin(angle)))

        # Draw the capsule
#         frame = cv2.line(frame, start_point, end_point, (0, 0, 255), 2)
#         frame = cv2.circle(frame, start_point, 5, (0, 0, 255), -1)
#         frame = cv2.circle(frame, end_point, 5, (0, 0, 255), -1)

        # Add text with transparent background
        text_str = f"Velocity: {desired_vel:.2f}\nAngle: {math.degrees(desired_angle_vel):.2f} degrees/sec"
        y0, dy = 30, 20
        for j, line in enumerate(text_str.split('\n')):
            y = y0 + j * dy
            frame = cv2.putText(frame, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
            frame = cv2.putText(frame, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)

        frames[i] = frame
    return frames

# Assuming you have frames and env
# frames = env.render(rollout, rollout_info, camera='hexapod_camera')
# media.show_video(frames, fps=1.0 / env.dt / 2)

# Call the add_capsule_and_text function with desired velocities and angles
# Replace the following lists with the actual desired_vels and desired_angle_vels lists used in your environment
desired_vels = [info['desired_vels'][info['current_idx']] for info in rollout_info]
desired_angle_vels = [info['desired_angle_vels'][info['current_idx']] for info in rollout_info]

# frames = env.render(rollout, rollout_info, camera='hexapod_camera')
frames = add_capsule_and_text(frames, desired_vels, desired_angle_vels)
media.show_video(frames, fps=1.0 / (eval_env.dt * eval_env.CPG_steps)/2)

In [ ]:
rng = jax.random.PRNGKey(1)
state = jit_reset(rng)
rollout = [state.pipeline_state]
rollout_info = [state.info]

# grab a trajectory
n_steps = 1000
render_every = 1

for i in range(n_steps):
  act_rng, rng = jax.random.split(rng)
  ctrl, _ = jit_inference_fn(state.obs, act_rng)
  # for i in range(env._physics_steps_per_control_step):
  state = jit_step(state, ctrl)
  rollout.append(state.pipeline_state)
#   print(state.pipeline_state.xd.ang[0], braxMath.rotate(state.pipeline_state.xd.ang[0], braxMath.quat_inv(state.pipeline_state.x.rot[0])),
#        '\n', state.pipeline_state.x.pos[0],braxMath.rotate(state.pipeline_state.x.pos[0], braxMath.quat_inv(state.pipeline_state.x.rot[0])),
#        '\n************************************************************************')
  rollout_info.append(state.info)

  if state.done:
    break

frames = env.render(rollout, camera='hexapod_camera')

In [ ]:
desired_vels = [info['desired_vels'][info['current_idx']] for info in rollout_info]
desired_angle_vels = [info['desired_angle_vels'][info['current_idx']] for info in rollout_info]

# frames = env.render(rollout, rollout_info, camera='hexapod_camera')
frames = add_capsule_and_text(frames, desired_vels, desired_angle_vels)
media.show_video(frames, fps=1.0 / (eval_env.dt * eval_env.CPG_steps)/2)